# <center> Introduction
Туториал вдохновлен [kernels](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline) и являеться его воплощением на примере [QR кодов](https://www.kaggle.com/c/receipt-categorisation)
Далее будем расматривать что такое NBSVM (Naive Bayes - Support Vector Machine) и как его применяют для линейных моделей.

NBSVM представлена Sida Wang и Chris Manning в работе: [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf).

Naive Bayes (NB) и Support Vector Machine (SVM) модель, которую используют для улучшения базового линейного метода для категоризации.

### <center>Немного теории из статьи:
 
Базовый линейный метод имеет математическую формулировку:
    $y = w_0 + \sum_{i=1}^m w_i x_i$
Задача линейной модели найти коэффициенты $w_i$ при $x_i$.

Naive Bayes - Support Vector Machine выполняет вспомагательную функцию (что указано в названии). Суть метода преобразовать коэффициенты $x_i$ так, что бы они были больше для целевой категории и поменьше для других (метод основан на бинарном представлении класса как One vs All). Для этого предлагаеться $x_i$ домножить на коэффициент важности переменной (в тексте для токена). Важность токена выводиться из теорема Байеса и в конечном виде имеет вид:
$$\large\begin{array}
\mathcal{r}&=& \log\left(\frac{p/\left\|p\right\|_1}{q/\left\| q\right\|_1}\right)
\end{array}$$


Где:
$\begin{array}\mathcal{p}&=& \alpha + \sum_{i : y^\left(i\right) = 1} f^\left(i\right)\end{array}$ и
$\begin{array}\mathcal{q}&=& \alpha + \sum_{i : y^\left(i\right) = 0} f^\left(i\right)\end{array}$

$\alpha$ - коэффициент сглаживания, для удобства равен 1 (и так исбегаем деления на 0). $p$ то как часто данный $x_i$ встречаеться при $y = 1$, $q$ - при $y = 0$.

Перейдем к воблощению метода в коде:

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import log_loss

## Read data

In [2]:
train = pd.read_csv('../../data/train.csv.gz')
train.fillna('', inplace=True)

In [3]:
vectorizer = CountVectorizer()
labeler = LabelEncoder()
lb = LabelBinarizer()

In [4]:
train.head(3)

,check_id,name,category,price,count
0,0,*3479755 ТRUF.Конф.кр.корп.гл.вк.шок180г,Чай и сладкое,49.0,2.0
1,0,3408392 ECONTA Мешки д/мусора 30л 30шт,Для дома,21.0,1.0
2,0,3260497 ЯШКИНО Рулет С ВАР.СГУЩ. 200г,Чай и сладкое,39.0,1.0


In [5]:
X = vectorizer.fit_transform(train.name.values)

In [6]:
y = labeler.fit_transform(train.category)


Разбиваем обучающую выборку на части. Для демонстрации NB-SVM будем брать только первый fold.


In [7]:
gkf = list(GroupKFold(n_splits=3).split(X, y, train.check_id.values))

Создадим матрицу в бинарном представлении категорий:

In [8]:
labels = lb.fit_transform(y)

In [9]:
logist = LogisticRegression()
mnb = MultinomialNB()

Быстрая проверка какие результаты дают LogisticRegression и MultinomialNB.

In [10]:
%%time
score = cross_val_score(logist, X, y, scoring='neg_log_loss', cv = gkf, n_jobs = -1)
print("%.3f +- %.4f" % (-np.mean(score), np.std(score)))

0.789 +- 0.0124
Wall time: 6.7 s


In [11]:
print('Результаты для первого фолда методом LogisticRegression = %.3f' % (-score[0]))

Результаты для первого фолда методом LogisticRegression = 0.799


In [12]:
%%time
score = cross_val_score(mnb, X, y, scoring='neg_log_loss', cv = gkf, n_jobs = -1)
print("%.3f +- %.4f" % (-np.mean(score), np.std(score)))

0.744 +- 0.0214
Wall time: 5.07 s


In [13]:
print('Результаты для первого фолда методом MultinomialNB = %.3f' % (-score[0]))

Результаты для первого фолда методом MultinomialNB = 0.762


Как видим оба метода примерно одинаковы по результатам (MultinomialNB слегка лучше).

Разделим train на X_train и X_valid, взяв за разбиение первый фолд созданый GroupKFold


#### Use 1st slit for example NB - SVM

In [14]:
X_train, X_valid, y_train, y_valid = X[gkf[0][0]], X[gkf[0][1]], labels[gkf[0][0], :], labels[gkf[0][1],:]

In [15]:
# Разбивка на 3 части : у нас 0,66 train part и 0,33 valid part
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((9121, 15758), (4561, 15758), (9121, 25), (4561, 25))

Кратко о MultinomialNB. Модель создает матрицу в которой указываеться, как часто каждая из характеристик (у нас тут tokens из слов) встречаеться при конкретном классе. В формуле ratio у нас под логорифмом дробь, по правилам логарифмирования это разница логарифмов.
$$\begin{array}
\mathcal{r}&=& \log\left(\frac{p/\left\|p\right\|_1}{q/\left\| q\right\|_1}\right)
&=& \log\left(p/\left\|p\right\|_1\right) - \log\left(q/\left\|q\right\|_1\right)
\end{array}$$
У MultinomialNB есть метод который вернет нам эти слогаемые: feature_log_prob_$[1]$ и feature_log_prob_$[0]$. Остаеться применить полученые коэффициенты к data.



In [16]:
preds = []
for idx, category in enumerate(labeler.classes_):
    #print (' Prediction for %s' % category)
    mnb.fit(X_train, y_train[:, idx])
    ratio = mnb.feature_log_prob_[1] - mnb.feature_log_prob_[0]
    logist.fit(X_train.multiply(ratio), y_train[:, idx])
    preds.append(logist.predict_proba(X_valid.multiply(ratio))[:,1])

In [17]:
print('Score after applying NB-SVM on data = %.3f' % (log_loss(y_valid, np.array(preds).T)))

Score after applying NB-SVM on data = 0.650


Результат NB-SVM лечше отдельно каждого их методов 0,799 и 0,744. После применения имеем 0,650.